In [1]:
import gc
gc.enable()

import sys
sys.path.append("../input/tez-lib/tez-lib")

import os

!nvidia-smi
import torch
torch.cuda.is_available()

import numpy as np
import pandas as pd
import tez
import torch
import torch.nn as nn
from joblib import Parallel, delayed
from transformers import AutoConfig, AutoModel, AutoTokenizer, BigBirdTokenizer, BigBirdModel
from torch.optim import Adam
from tqdm import tqdm

In [2]:
target_id_map = {
    "B-Lead": 0,
    "I-Lead": 1,
    "B-Position": 0,
    "I-Position": 1,
    "B-Evidence": 0,
    "I-Evidence": 1,
    "B-Claim": 0,
    "I-Claim": 1,
    "B-Concluding Statement": 0,
    "I-Concluding Statement": 1,
    "B-Counterclaim": 0,
    "I-Counterclaim": 1,
    "B-Rebuttal": 0,
    "I-Rebuttal": 1,
    "O": 2,
    "PAD": -100,
}


id_target_map = {v: k for k, v in target_id_map.items()}

class args1:
    input_path = "../input/feedback-prize-2021/"
    model = "../input/longformer4096/allenai--longformer-base-4096.main.e351d9d5da3eed48886f39eed7b64014debe4925/"
    #model = "../input/deberta-base/microsoft--deberta-base.main.7d4c0126b06bd59dccd3e48e467ed11e37b77f3f/"

    tez_model= "../input/longformer-trained/"
    #tez_model = "../input/deberta/"
    output = "."
    batch_size = 8
    max_len = 4096

In [3]:
class FeedbackDataset:
    def __init__(self, samples, max_len, tokenizer):
        self.samples = samples
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.length = len(samples)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        input_ids = self.samples[idx]["input_ids"]
        # print(input_ids)
        # print(input_labels)

        # add start token id to the input_ids
        input_ids = [self.tokenizer.cls_token_id] + input_ids

        if len(input_ids) > self.max_len - 1:
            input_ids = input_ids[: self.max_len - 1]

        # add end token id to the input_ids
        input_ids = input_ids + [self.tokenizer.sep_token_id]
        attention_mask = [1] * len(input_ids)

        # padding_length = self.max_len - len(input_ids)
        # if padding_length > 0:
        #     if self.tokenizer.padding_side == "right":
        #         input_ids = input_ids + [self.tokenizer.pad_token_id] * padding_length
        #         attention_mask = attention_mask + [0] * padding_length
        #     else:
        #         input_ids = [self.tokenizer.pad_token_id] * padding_length + input_ids
        #         attention_mask = [0] * padding_length + attention_mask

        # return {
        #     "ids": torch.tensor(input_ids, dtype=torch.long),
        #     "mask": torch.tensor(attention_mask, dtype=torch.long),
        # }

        return {
            "ids": input_ids,
            "mask": attention_mask,
        }

In [4]:
class Collate:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, batch):
        output = dict()
        output["ids"] = [sample["ids"] for sample in batch]
        output["mask"] = [sample["mask"] for sample in batch]

        # calculate max token length of this batch
        batch_max = max([len(ids) for ids in output["ids"]])

        # add padding
        if self.tokenizer.padding_side == "right":
            output["ids"] = [s + (batch_max - len(s)) * [self.tokenizer.pad_token_id] for s in output["ids"]]
            output["mask"] = [s + (batch_max - len(s)) * [0] for s in output["mask"]]
        else:
            output["ids"] = [(batch_max - len(s)) * [self.tokenizer.pad_token_id] + s for s in output["ids"]]
            output["mask"] = [(batch_max - len(s)) * [0] + s for s in output["mask"]]

        # convert to tensors
        output["ids"] = torch.tensor(output["ids"], dtype=torch.long)
        output["mask"] = torch.tensor(output["mask"], dtype=torch.long)

        return output

In [5]:
class FeedbackModel(tez.Model):
    def __init__(self, model_name, num_labels):
        super().__init__()
        self.model_name = model_name
        self.num_labels = num_labels
        config = AutoConfig.from_pretrained(model_name)

        hidden_dropout_prob: float = 0.1
        layer_norm_eps: float = 1e-7
        config.update(
            {
                "output_hidden_states": True,
                "hidden_dropout_prob": hidden_dropout_prob,
                "layer_norm_eps": layer_norm_eps,
                "add_pooling_layer": False,
            }
        )
        self.transformer = AutoModel.from_config(config)
        self.output = nn.Linear(config.hidden_size, self.num_labels)

    def forward(self, ids, mask):
        transformer_out = self.transformer(ids, mask)
        sequence_output = transformer_out.last_hidden_state
        logits = self.output(sequence_output)
        logits = torch.softmax(logits, dim=-1)
        return logits, 0, {}

In [6]:
def _prepare_test_data_helper(args, tokenizer, ids):
    test_samples = []
    for idx in ids:
        filename = os.path.join(args.input_path, "test", idx + ".txt")
        with open(filename, "r") as f:
            text = f.read()

        encoded_text = tokenizer.encode_plus(
            text,
            add_special_tokens=False,
            return_offsets_mapping=True,
        )
        input_ids = encoded_text["input_ids"]
        offset_mapping = encoded_text["offset_mapping"]

        sample = {
            "id": idx,
            "input_ids": input_ids,
            "text": text,
            "offset_mapping": offset_mapping,
        }

        test_samples.append(sample)
    return test_samples


def prepare_test_data(df, tokenizer, args):
    test_samples = []
    ids = df["id"].unique()
    ids_splits = np.array_split(ids, 4)
    #print(ids)
#     results = Parallel(n_jobs=4, backend="multiprocessing")(
#         delayed(_prepare_test_data_helper)(args, tokenizer, idx) for idx in ids_splits
#     )
    results = []
    for idx in ids_splits:
        results.append(_prepare_test_data_helper(args, tokenizer, idx))
   #########     
    for result in results:
        test_samples.extend(result)

    return test_samples

In [7]:
files = os.listdir('../input/feedback-prize-2021/test/')
IDS = np.array([f.replace('.txt','') for f in files if 'txt' in f])

df = pd.DataFrame({"id": IDS,"class": np.nan, "predictionstring": np.nan})

#tokenizer = AutoTokenizer.from_pretrained('../input/deberta-base/microsoft--deberta-base.main.7d4c0126b06bd59dccd3e48e467ed11e37b77f3f/')
tokenizer = AutoTokenizer.from_pretrained('../input/longformer4096/allenai--longformer-base-4096.main.e351d9d5da3eed48886f39eed7b64014debe4925/')

collate = Collate(tokenizer=tokenizer)

test_samples = prepare_test_data(df, tokenizer, args1)

raw_preds = []
# for fold_ in range(10):
#     current_idx = 0
#     test_dataset = FeedbackDataset(test_samples, args1.max_len, tokenizer)
    
#     if fold_ < 5:
#         model = FeedbackModel(model_name=args1.model, num_labels=len(target_id_map) - 1)
#         model.load(os.path.join(args1.tez_model, f"model_{fold_}.bin"), weights_only=True)
#         preds_iter = model.predict(test_dataset, batch_size=args1.batch_size, n_jobs=-1, collate_fn=collate)
#     else:
#         model = FeedbackModel(model_name=args2.model, num_labels=len(target_id_map) - 1)
#         model.load(os.path.join(args2.tez_model, f"model_{fold_-5}.bin"), weights_only=True)
#         preds_iter = model.predict(test_dataset, batch_size=args2.batch_size, n_jobs=-1, collate_fn=collate)

fold_ = 0
current_idx = 0
test_dataset = FeedbackDataset(test_samples, args1.max_len, tokenizer)

model = FeedbackModel(model_name=args1.model, num_labels=len(target_id_map) - 1)
model.load(os.path.join(args1.tez_model, f"model_{fold_}_longformer.bin"), weights_only=True)
#model.load(os.path.join(args1.tez_model, f"model_{fold_}_deberta.bin"), weights_only=True)

preds_iter = model.predict(test_dataset, batch_size=args1.batch_size, n_jobs=-1, collate_fn=collate)

current_idx = 0

for preds in preds_iter:
    preds = preds.astype(np.float16)
    # preds = preds / 10
    if fold_ == 0:
        raw_preds.append(preds)
    else:
        raw_preds[current_idx] += preds
        current_idx += 1
torch.cuda.empty_cache()
gc.collect()

In [8]:
final_preds = []
final_scores = []

for rp in raw_preds:
    pred_class = np.argmax(rp, axis=2)
    pred_scrs = np.max(rp, axis=2)
    print(pred_class, pred_scrs)
    for pred, pred_scr in zip(pred_class, pred_scrs):
        pred = pred.tolist()
        pred_scr = pred_scr.tolist()
        final_preds.append(pred)
        final_scores.append(pred_scr)

for j in range(len(test_samples)):
    tt = [id_target_map[p] for p in final_preds[j][1:]]
    tt_score = final_scores[j][1:]
    test_samples[j]["preds"] = tt
    test_samples[j]["pred_scores"] = tt_score

In [9]:
def jn(pst, start, end):
    return " ".join([str(x) for x in pst[start:end]])


def link_evidence(oof):
    thresh = 1
    idu = oof['id'].unique()
    idc = idu[1]
    eoof = oof[oof['class'] == "Evidence"]
    neoof = oof[oof['class'] != "Evidence"]
    for thresh2 in range(26,27, 1):
        retval = []
        for idv in idu:
            for c in  ['Lead', 'Position', 'Evidence', 'Claim', 'Concluding Statement',
                   'Counterclaim', 'Rebuttal']:
                q = eoof[(eoof['id'] == idv) & (eoof['class'] == c)]
                if len(q) == 0:
                    continue
                pst = []
                for i,r in q.iterrows():
                    pst = pst +[-1] + [int(x) for x in r['predictionstring'].split()]
                start = 1
                end = 1
                for i in range(2,len(pst)):
                    cur = pst[i]
                    end = i
                    #if pst[start] == 205:
                    #   print(cur, pst[start], cur - pst[start])
                    if (cur == -1 and c != 'Evidence') or ((cur == -1) and ((pst[i+1] > pst[end-1] + thresh) or (pst[i+1] - pst[start] > thresh2))):
                        retval.append((idv, c, jn(pst, start, end)))
                        start = i + 1
                v = (idv, c, jn(pst, start, end+1))
                #print(v)
                retval.append(v)
        roof = pd.DataFrame(retval, columns = ['id', 'class', 'predictionstring']) 
        roof = roof.merge(neoof, how='outer')
        return roof

In [29]:
proba_thresh = {
    "Lead": 0.7,
    "Position": 0.55,
    "Evidence": 0.65,
    "Claim": 0.55,
    "Concluding Statement": 0.7,
    "Counterclaim": 0.5,
    "Rebuttal": 0.55,
}

min_thresh = {
    "Lead": 9,
    "Position": 5,
    "Evidence": 14,
    "Claim": 3,
    "Concluding Statement": 11,
    "Counterclaim": 6,
    "Rebuttal": 4,
}

submission = []
for sample_idx, sample in enumerate(test_samples):
    preds = sample["preds"]
    offset_mapping = sample["offset_mapping"]
    sample_id = sample["id"]
    sample_text = sample["text"]
    sample_input_ids = sample["input_ids"]
    sample_pred_scores = sample["pred_scores"]
    sample_preds = []

    if len(preds) < len(offset_mapping):
        preds = preds + ["O"] * (len(offset_mapping) - len(preds))
        sample_pred_scores = sample_pred_scores + [0] * (len(offset_mapping) - len(sample_pred_scores))
    
    idx = 0
    phrase_preds = []
    while idx < len(offset_mapping):
        start, _ = offset_mapping[idx]
        if preds[idx] != "O":
            label = preds[idx][2:]
        else:
            label = "O"
        phrase_scores = []
        phrase_scores.append(sample_pred_scores[idx])
        idx += 1
        while idx < len(offset_mapping):
            if label == "O":
                matching_label = "O"
            else:
                matching_label = f"I-{label}"
            if preds[idx] == matching_label:
                _, end = offset_mapping[idx]
                phrase_scores.append(sample_pred_scores[idx])
                idx += 1
            else:
                break
        if "end" in locals():
            phrase = sample_text[start:end]
            phrase_preds.append((phrase, start, end, label, phrase_scores))

    temp_df = []
    for phrase_idx, (phrase, start, end, label, phrase_scores) in enumerate(phrase_preds):
        word_start = len(sample_text[:start].split())
        word_end = word_start + len(sample_text[start:end].split())
        word_end = min(word_end, len(sample_text.split()))
        ps = " ".join([str(x) for x in range(word_start, word_end)])
        if label != "O":
            if sum(phrase_scores) / len(phrase_scores) >= proba_thresh[label]:
                if len(ps.split()) >= min_thresh[label]:
                    temp_df.append((sample_id, phrase, ps))
    
    temp_df = pd.DataFrame(temp_df, columns=["id", "class", "predictionstring"])
    submission.append(temp_df)

submission = pd.concat(submission).reset_index(drop=True)
submission = link_evidence(submission)

In [30]:
submission.rename(columns={"class": "text"}, inplace=True)
temp = submission['predictionstring'].values
temp = list(map(str, temp))
submission['start'] = [int(x.split(' ')[0]) for x in temp]
submission = submission[["id", "text", "predictionstring"]]

In [31]:
submission

In [21]:
# class sub_Dataset(torch.utils.data.Dataset):

#     def __init__(self, df):

#         self.texts = [tokenizer(text, 
#                                padding='max_length', max_length = 256, truncation=True,
#                                 return_tensors="pt") for text in df['text']]
#         self.prev = [one_hot for one_hot in df['one_hot']]

#     def get_batch_texts(self, idx):
#         # Fetch a batch of inputs
#         return self.texts[idx]
    
#     def get_batch_prev(self, idx):
#         return self.prev[idx]
    
#     def __getitem__(self, idx):

#         batch_texts = self.get_batch_texts(idx)
#         batch_prev = self.get_batch_prev(idx)

#         return batch_texts, batch_prev

In [23]:
labels = {'Lead':0,
          'Position':1,
          'Claim':2,
          'Concluding Statement':3,
          'Evidence':4,
          'Counterclaim':5,
          'Rebuttal':6
          }
prev_label = {'Lead':0,
          'Position':1,
          'Claim':2,
          'Concluding Statement':3,
          'Evidence':4,
          'Counterclaim':5,
          'Rebuttal':6,
          'Start':7
          }

In [ ]:
# sequence = submission['predictionstring']
# sequence = sequence.apply(lambda x: x.split())
# sequence = sequence.apply(lambda x: list(map(int, x)))
# sequence
# submission_new = submission
# start, end = [],[]
# for i in range(len(sequence)):
#     start.append(sequence[i][0])
#     end.append(sequence[i][-1])
# submission_new['Start'] = start
# submission_new['End'] = end

# prev = [7]

# for i in range(1, len(sequence)):
#     if submission_new['End'][i-1] < submission_new['Start'][i]:
#         prev.append(prev_label[submission['discourse_type'][i-1]])
#     else:
#         prev.append(7)

# df_new['one_hot'] = prev

In [14]:
tokenizer = AutoTokenizer.from_pretrained('../input/bigbird-roberta-base/google--bigbird-roberta-base.main.5a145f7852cba9bd431386a58137bf8a29903b90/')

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()
        
        self.bert = BigBirdModel.from_pretrained('../input/bigbird-roberta-base/google--bigbird-roberta-base.main.5a145f7852cba9bd431386a58137bf8a29903b90/')
        #self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 7)
        self.linear_2 = nn.Linear(15,7)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask, prev_identity):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        linear_output = torch.cat((linear_output, prev_identity), 1)
        linear_output = self.linear_2(linear_output)
        final_layer = self.relu(linear_output)

        return final_layer
    
model = BertClassifier()
model.load_state_dict(torch.load('../input/c-bigbird-trained/bigbird-roberta-base'))

In [49]:
# test = sub_Dataset(submission)
# test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

def test_data(test,temp):
    test_new = {}
    test_new['text'] = tokenizer(test['text'], padding='max_length', max_length = 256, truncation=True,return_tensors="pt")
    if test['id'] == temp[1]:
        test_new['prev'] = torch.Tensor([temp[0]]).to(torch.int64)
    else: 
        test_new['prev'] = torch.Tensor([7]).to(torch.int64)
    test_new['id'] = test['id']
    return test_new

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

if use_cuda:
    model = model.cuda()
    
    temp = [torch.Tensor([1]).to(torch.int64),-1]
    output_label = []
    with torch.no_grad():

        for _, data in submission.iterrows():
            
            test_input = test_data(data,temp)
            mask = test_input['text']['attention_mask'].to(device)
            input_id = test_input['text']['input_ids'].squeeze(1).to(device)
            prev = test_input['prev']
            prev = torch.nn.functional.one_hot(prev, num_classes=8).to(device)
            
            output = model(input_id, mask, prev)
            label = output.argmax(dim=1)
            
            list_of_key = list(prev_label.keys())
            list_of_value = list(prev_label.values())
            
            position = list_of_value.index(label)
            label = list_of_key[position]
            
            output_label.append(label)
            temp = [position, test_input['id']]

submission['class'] = output_label
submission = submission[['id', 'class', 'predictionstring']]
submission

In [54]:
submission.to_csv("submission.csv",index=False)